## [Camera Calibration and 3D Reconstruction](https://docs.opencv.org/3.4/d9/db7/tutorial_py_table_of_contents_calib3d.html)
이 섹션에서는 카메라 보정, 스테레오 이미징 등을 배우게됩니다.

### [Camera Calibration](https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html)
Let's find how good is our camera. Is there any distortion in images taken with it? If so how to correct it?

#### Goal
이 섹션에서는

* 카메라로 인한 왜곡 유형
* 카메라의 본질적 및 외적 속성을 찾는 방법
* 이러한 속성을 기반으로 이미지를 왜곡되지 않게 방법

#### Basics
일부 핀홀 카메라는 이미지에 큰 왜곡을 일으 킵니다. 두 가지 종류의 왜곡은 **방사형 왜곡**과 **접선 왜곡**입니다.

방사형 왜곡은 직선이 곡선으로 나타나는 원인이됩니다. 방사형 왜곡은 이미지 중심에서 먼 지점이 클수록 커집니다. 예를 들어 체스 보드의 두 가장자리에 빨간색 선이 표시되는 이미지가 아래에 나와 있습니다. 그러나 체스 판의 경계선이 직선이 아니고 붉은 선과 일치하지 않는 것을 볼 수 있습니다. 예상되는 모든 직선이 돌출되어 있습니다. 자세한 내용은 [왜곡(광학)](https://en.wikipedia.org/wiki/Distortion_%28optics%29) 을 방문 하십시오.

![](calib_radial.jpg) 

Radial distortion can be represented as follows:

\\(x_{distorted} = x( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\ y_{distorted} = y( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6)\\)

마찬가지로, 접선 왜곡은 이미지 촬영 렌즈가 이미징 평면과 완벽하게 평행하게 정렬되어 있지 않기 때문에 발생합니다. 따라서 이미지의 일부 영역이 예상보다 가깝게 보일 수 있습니다. 접선 왜곡의 크기는 다음과 같이 나타낼 수 있습니다.

\\(x_{distorted} = x + [ 2p_1xy + p_2(r^2+2x^2)] \\ y_{distorted} = y + [ p_1(r^2+ 2y^2)+ 2p_2xy]\\)

In short, we need to find five parameters, known as distortion coefficients given by:

\\(Distortion \; coefficients=(k_1 \hspace{10pt} k_2 \hspace{10pt} p_1 \hspace{10pt} p_2 \hspace{10pt} k_3)\\)


이 외에도 카메라의 내장 및 외장 매개 변수와 같은 다른 정보가 필요합니다. 내장 매개 변수는 카메라에만 적용됩니다. 초점 거리 \\(((f_x, f_y) \\)및 광학 중심 \\((c_x, c_y)\\)과 같은 정보가 포함됩니다. 초점 거리와 광학 중심을 사용하여 특정 카메라의 렌즈로 인한 왜곡을 제거하는 데 사용할 수있는 카메라 매트릭스를 만들 수 있습니다. 카메라 매트릭스는 특정 카메라에만 적용되므로 한 번 계산하면 동일한 카메라로 촬영 한 다른 이미지에 다시 사용할 수 있습니다. 그것은 3x3 매트릭스로 표현됩니다 :

\\(camera \; matrix = \left [ \begin{matrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{matrix} \right ]\\)

외재 매개 변수는 3D 점의 좌표를 좌표계로 변환하는 회전 및 변환 벡터에 해당합니다.

스테레오 어플리케이션의 경우 이러한 왜곡을 먼저 수정해야합니다. 이러한 매개 변수를 찾으려면 잘 정의 된 패턴 (예 : 체스 보드)의 샘플 이미지를 제공해야합니다. 우리는 상대 위치 (예 : 체스 보드의 사각형 코너)를 이미 알고있는 몇 가지 구체적인 점을 찾습니다. 우리는 실세계 공간에서이 점들의 좌표를 알고 있고 이미지의 좌표를 알고 있기 때문에 왜곡 계수를 풀 수 있습니다. 더 나은 결과를 얻으려면 적어도 10 개의 테스트 패턴이 필요합니다.

#### Code
앞서 언급했듯이 카메라 교정을 위해 최소한 10 가지 테스트 패턴이 필요합니다. OpenCV에는 체스 보드 이미지 (samples / data / left01.jpg - left14.jpg 참조)가 제공되므로이를 활용할 것입니다. 체스 보드의 이미지를 생각해보십시오. 카메라 보정에 필요한 중요한 입력 데이터는 3D 실세계 점 세트와 이미지에서 이러한 점의 해당 2D 좌표입니다. 2D 이미지 포인트는 이미지에서 쉽게 찾을 수 있습니다. (이 이미지 포인트는 두 개의 검정색 사각형이 체스 보드에서 서로 닿는 위치 임)

실제 공간에서 3D 점은 어떻습니까? 이러한 이미지는 고정 카메라에서 가져오고 체스 보드는 다른 위치와 방향에 배치됩니다. 그래서 \\((X, Y, Z)\\) 값을 알아야합니다. 그러나 간단히 말하면, 체스 판이 XY 평면에 고정되어 있다고 말할 수 있습니다 (따라서 Z = 0 항상). 따라서 카메라는 그에 따라 이동되었습니다. 이 고려 사항은 X, Y 값만 찾을 수있게 해줍니다. 이제 X, Y 값에 대해 포인트를 (0,0), (1,0), (2,0), ...으로 전달하면 포인트의 위치를 ​​나타낼 수 있습니다. 이 경우 우리가 얻는 결과는 체스 보드 스퀘어 크기의 척도가됩니다. 그러나 사각형 크기 (30mm)를 알면 값을 (0,0), (30,0), (60,0), ...으로 전달할 수 있습니다. 따라서 우리는 결과를 mm로 얻습니다. (이 경우 이미지를 찍지 않았으므로 정사각형 크기를 알지 못하므로 정사각형 크기로 전달합니다.)

3D 점을 객체 점이라고하고 2D 이미지 점을 이미지 점이라고합니다.

##### Serup
따라서 체스 보드에서 패턴을 찾으 려면 [cv.findChessboardCorners()](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a) 함수를 사용할 수 있습니다 . 8x8 그리드, 5x5 그리드 등 원하는 패턴을 전달해야합니다.이 예에서는 7x6 그리드를 사용합니다. (일반적으로 체스 보드는 8x8 사각형과 7x7 내부 코너를가집니다.) 패턴을 얻으면 True가 될 모서리 점과 retval을 반환합니다. 이 모퉁이는 순서대로 배치됩니다 (왼쪽에서 오른쪽, 위에서 아래로)

##### See also
> 이 기능은 모든 이미지에서 필요한 패턴을 찾지 못할 수도 있습니다. 따라서 좋은 방법은 카메라를 시작하고 각 프레임에서 필요한 패턴을 확인하도록 코드 를 작성 하는 것입니다. 패턴을 얻은 후에 모서리를 찾아 목록에 저장하십시오. 또한 체스 보드를 다른 방향으로 조정할 수 있도록 다음 프레임을 읽기 전에 일정 간격을 두어 배치하십시오. 필요한 수의 좋은 패턴이 얻어 질 때까지이 과정을 계속하십시오. 여기에 제공된 예에서도 14 개의 주어진 이미지 중에서 얼마나 많은 이미지가 좋은지 확실하지 않습니다. 따라서 우리는 모든 이미지를 읽고 좋은 이미지 만 가져야합니다.
체스 판 대신에 원형 격자를 사용할 수도 있습니다. 이 경우 패턴을 찾으려면 [cv.findCirclesGrid()](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#gad1205c4b803a21597c7d6035f5efd775) 함수를 사용해야합니다 . 원형 그리드를 사용하여 카메라 보정을 수행하는 데 필요한 이미지가 충분하지 않습니다.

모서리를 찾으면 [cv.cornerSubPix()](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e)를 사용하여 정확도를 높일 수 있습니다 . 또한 [cv.drawChessboardCorners()](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga6a10b0bb120c4907e5eabbcd22319022)를 사용하여 패턴을 그릴 수 있습니다 . 이 모든 단계는 아래 코드에 포함되어 있습니다.

In [1]:
#%%writefile CalibrationCode.py

import numpy as np
import cv2 as cv
import glob

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./07CalibrationData/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

One image with pattern drawn on it is shown below:

![](calib_pattern.jpg)

##### Calibration
이제 물점과 이미지 점이 생겨 교정 준비가 끝났습니다. 카메라 매트릭스, 왜곡 계수, 회전 및 변환 벡터 등을 반환 하는 함수 [cv.calibrateCamera()](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d) 를 사용할 수 있습니다 .


In [2]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
np.savez('B.npz', mtx = mtx, dist = dist, rvecs = rvecs, tvecs = tvecs)

##### Undistortion
자, 우리는 이미지를 가져 와서 그것을 왜곡시킬 수있다. OpenCV에는이를 수행하는 두 가지 방법이 있습니다. 그러나 먼저 cv.getOptimalNewCameraMatrix ()를 사용하여 자유 스케일링 매개 변수를 기반으로 카메라 행렬을 수정할 수 있습니다 . 스케일 파라미터 alpha = 0의 경우, 불필요한 최소의 픽셀을 가지는 왜곡이없는 이미지를 돌려줍니다. 따라서 이미지 모서리에서 일부 픽셀을 제거 할 수도 있습니다. 알파 = 1이면 모든 픽셀이 검정색 이미지가 약간 남습니다. 이 함수는 결과를 자르기 위해 사용할 수있는 이미지 ROI도 반환합니다.

그래서 우리는 새로운 이미지를 찍습니다 (이 경우 left12.jpg이 장의 첫 번째 이미지입니다).

In [3]:
img = cv.imread('./07CalibrationData/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

1. Using cv.undistort()

This is the easiest way. Just call the function and use ROI obtained above to crop the result.

In [4]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y + h, x:x + w]
cv.imwrite('calibresult.png', dst)

True

2. Using remapping

이 방법은 조금 더 어렵습니다. 먼저 왜곡 된 이미지에서 왜곡되지 않은 이미지까지 매핑 기능을 찾습니다. 그런 다음 다시 매핑 기능을 사용하십시오.

In [5]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y + h, x:x + w]
cv.imwrite('calibresult.png', dst)

True

여전히 두 가지 방법 모두 동일한 결과를 제공합니다. 아래 결과를보십시오 :
![](calibresult.png)
결과에서 모든 가장자리가 직선임을 알 수 있습니다.

이제 NumPy (np.savez, np.savetxt 등)의 쓰기 함수를 사용하여 향후 사용을 위해 카메라 행렬 및 왜곡 계수를 저장할 수 있습니다.

#### Re-projection Error
 투영 오차는 발견 된 매개 변수가 얼마나 정확한지를 잘 예측합니다. 재 투영 오차가 0에 가까울수록 우리가 찾은 매개 변수가 더 정확합니다. 고유, 왜곡, 회전 및 변환 행렬이 주어지면 먼저 cv.projectPoints ()를 사용하여 객체 점을 이미지 점으로 변환해야합니다 . 그런 다음 변환을 통해 얻은 것과 코너 찾기 알고리즘 사이의 절대 표준을 계산할 수 있습니다. 평균 오차를 찾기 위해 모든 보정 영상에 대해 계산 된 오차의 산술 평균을 계산합니다.

In [6]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error         = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error   += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.023686000375385676


#### Additional Resources

#### Exercises
Try camera calibration with circular grid.